In [96]:
# Imports 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import zipfile

import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
# Note: the statsmodels import may print out a 'FutureWarning'. Thats fine.

In [97]:
zf = zipfile.ZipFile('config_files.zip') # having First.csv zipped file.
data = pd.read_csv(zf.open('FFG_Hackathon_Country_Level_Data.csv'))

In [98]:
data

,Year,Country Name,Country Code,AG.LND.AGRI.K2,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EG.ELC.COAL.ZS,EG.ELC.FOSL.ZS,EG.ELC.HYRO.ZS,EG.ELC.NGAS.ZS,...,EN.ATM.CO2E.SF.ZS,EN.CO2.BLDG.ZS,EN.CO2.ETOT.ZS,EN.CO2.MANF.ZS,EN.CO2.OTHX.ZS,EN.CO2.TRAN.ZS,EN.POP.DNST,NY.GDP.TOTL.RT.ZS,SP.RUR.TOTL,SP.URB.TOTL
0,2007,Albania,ALB,11190.00,27400.0,28750.0,0.000000,2.517483,97.482517,0.000000,...,1.867414,7.407407,8.730159,16.931217,11.640212,55.555556,108.394781,1.761086,1517619.0,1452398.0
1,2007,Algeria,DZA,412520.00,2381740.0,2381740.0,0.000000,99.392408,0.607592,97.257770,...,2.755153,16.028285,40.011786,11.455510,0.000000,32.516205,14.345383,32.464934,11839539.0,22327433.0
2,2007,Angola,AGO,508468.17,1246700.0,1246700.0,0.000000,22.381100,77.618900,0.000000,...,0.000000,23.858921,12.344398,23.547718,0.311203,40.041494,16.768559,46.279012,8879762.0,12025601.0
3,2007,Argentina,ARG,1440330.00,2736690.0,2780400.0,2.179398,63.438998,28.270726,51.046874,...,2.091227,14.616174,33.816513,20.510962,5.405569,25.644742,14.500837,4.995813,3823185.0,35861110.0
4,2007,Armenia,ARM,17519.00,28470.0,29740.0,0.000000,25.245846,31.417430,25.245846,...,0.145033,22.291667,22.500000,24.375000,8.750000,22.083333,103.007306,0.009296,1063397.0,1869221.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,2016,"Venezuela, RB",VEN,215000.00,882050.0,912050.0,NaN,NaN,NaN,NaN,...,0.285899,NaN,NaN,NaN,NaN,NaN,33.837287,NaN,3532295.0,26313884.0
1306,2016,Vietnam,VNM,121722.00,310070.0,331230.0,NaN,NaN,NaN,NaN,...,42.138520,NaN,NaN,NaN,NaN,NaN,301.992208,3.355316,61324000.0,32314724.0
1307,2016,"Yemen, Rep.",YEM,233858.00,527970.0,527970.0,NaN,NaN,NaN,NaN,...,3.041825,NaN,NaN,NaN,NaN,NaN,51.457867,0.797530,17552294.0,9615916.0
1308,2016,Zambia,ZMB,238360.00,743390.0,752610.0,NaN,NaN,NaN,NaN,...,16.619116,NaN,NaN,NaN,NaN,NaN,22.012009,11.680386,9419162.0,6944345.0


In [99]:
data.shape

(1310, 38)

Looking at the dataframe, we notice that in the year 2016, there are no recorded values for a good majority of the columns. Hence, we break up the dataframe into two separate dataframes: pre-2016 and 2016. 

In [100]:
data_till_2016 = data[:-131]
data_till_2016

,Year,Country Name,Country Code,AG.LND.AGRI.K2,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EG.ELC.COAL.ZS,EG.ELC.FOSL.ZS,EG.ELC.HYRO.ZS,EG.ELC.NGAS.ZS,...,EN.ATM.CO2E.SF.ZS,EN.CO2.BLDG.ZS,EN.CO2.ETOT.ZS,EN.CO2.MANF.ZS,EN.CO2.OTHX.ZS,EN.CO2.TRAN.ZS,EN.POP.DNST,NY.GDP.TOTL.RT.ZS,SP.RUR.TOTL,SP.URB.TOTL
0,2007,Albania,ALB,11190.00,27400.0,28750.0,0.000000,2.517483,97.482517,0.000000,...,1.867414,7.407407,8.730159,16.931217,11.640212,55.555556,108.394781,1.761086,1517619.0,1452398.0
1,2007,Algeria,DZA,412520.00,2381740.0,2381740.0,0.000000,99.392408,0.607592,97.257770,...,2.755153,16.028285,40.011786,11.455510,0.000000,32.516205,14.345383,32.464934,11839539.0,22327433.0
2,2007,Angola,AGO,508468.17,1246700.0,1246700.0,0.000000,22.381100,77.618900,0.000000,...,0.000000,23.858921,12.344398,23.547718,0.311203,40.041494,16.768559,46.279012,8879762.0,12025601.0
3,2007,Argentina,ARG,1440330.00,2736690.0,2780400.0,2.179398,63.438998,28.270726,51.046874,...,2.091227,14.616174,33.816513,20.510962,5.405569,25.644742,14.500837,4.995813,3823185.0,35861110.0
4,2007,Armenia,ARM,17519.00,28470.0,29740.0,0.000000,25.245846,31.417430,25.245846,...,0.145033,22.291667,22.500000,24.375000,8.750000,22.083333,103.007306,0.009296,1063397.0,1869221.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,2015,"Venezuela, RB",VEN,215000.00,882050.0,912050.0,0.000000,36.304958,63.695042,19.419168,...,0.300794,NaN,NaN,NaN,NaN,NaN,34.104449,NaN,3563493.0,26518336.0
1175,2015,Vietnam,VNM,121478.00,310070.0,331230.0,29.571446,63.267942,36.613975,33.210467,...,44.209276,NaN,NaN,NaN,NaN,NaN,298.890818,4.206113,61343883.0,31333193.0
1176,2015,"Yemen, Rep.",YEM,234326.00,527970.0,527970.0,0.000000,100.000000,0.000000,39.804732,...,2.700445,NaN,NaN,NaN,NaN,NaN,50.188247,2.105149,17282718.0,9215171.0
1177,2015,Zambia,ZMB,238360.00,743390.0,752610.0,0.000000,3.006176,96.993824,0.000000,...,10.679612,NaN,NaN,NaN,NaN,NaN,21.360741,11.163582,9224797.0,6654564.0


In [101]:
data_2016 = data[-131:]
data_2016

,Year,Country Name,Country Code,AG.LND.AGRI.K2,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EG.ELC.COAL.ZS,EG.ELC.FOSL.ZS,EG.ELC.HYRO.ZS,EG.ELC.NGAS.ZS,...,EN.ATM.CO2E.SF.ZS,EN.CO2.BLDG.ZS,EN.CO2.ETOT.ZS,EN.CO2.MANF.ZS,EN.CO2.OTHX.ZS,EN.CO2.TRAN.ZS,EN.POP.DNST,NY.GDP.TOTL.RT.ZS,SP.RUR.TOTL,SP.URB.TOTL
1179,2016,Albania,ALB,11817.0,27400.0,28750.0,NaN,NaN,NaN,NaN,...,0.323363,NaN,NaN,NaN,NaN,NaN,104.967190,1.333387,1195854.0,1680247.0
1180,2016,Algeria,DZA,413602.0,2381741.0,2381741.0,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,17.025950,12.032482,11573776.0,28977628.0
1181,2016,Angola,AGO,558423.8,1246700.0,1246700.0,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,23.135064,10.857114,10340319.0,18502165.0
1182,2016,Argentina,ARG,1487000.0,2736690.0,2780400.0,NaN,NaN,NaN,NaN,...,1.495228,NaN,NaN,NaN,NaN,NaN,15.928135,0.936008,3649822.0,39940546.0
1183,2016,Armenia,ARM,16768.0,28470.0,29740.0,NaN,NaN,NaN,NaN,...,0.071124,NaN,NaN,NaN,NaN,NaN,103.131226,0.221818,1083966.0,1852180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,2016,"Venezuela, RB",VEN,215000.0,882050.0,912050.0,NaN,NaN,NaN,NaN,...,0.285899,NaN,NaN,NaN,NaN,NaN,33.837287,NaN,3532295.0,26313884.0
1306,2016,Vietnam,VNM,121722.0,310070.0,331230.0,NaN,NaN,NaN,NaN,...,42.138520,NaN,NaN,NaN,NaN,NaN,301.992208,3.355316,61324000.0,32314724.0
1307,2016,"Yemen, Rep.",YEM,233858.0,527970.0,527970.0,NaN,NaN,NaN,NaN,...,3.041825,NaN,NaN,NaN,NaN,NaN,51.457867,0.797530,17552294.0,9615916.0
1308,2016,Zambia,ZMB,238360.0,743390.0,752610.0,NaN,NaN,NaN,NaN,...,16.619116,NaN,NaN,NaN,NaN,NaN,22.012009,11.680386,9419162.0,6944345.0


In [102]:
perc_miss = data_2016.isnull().sum() * 100 / len(data_2016)
missing_values = pd.DataFrame({'column_name': data_2016.columns,
                                 'percent_missing': perc_miss})
missing_values

,column_name,percent_missing
Year,Year,0.000000
Country Name,Country Name,0.000000
Country Code,Country Code,0.000000
AG.LND.AGRI.K2,AG.LND.AGRI.K2,0.000000
AG.LND.TOTL.K2,AG.LND.TOTL.K2,0.000000
AG.SRF.TOTL.K2,AG.SRF.TOTL.K2,0.000000
EG.ELC.COAL.ZS,EG.ELC.COAL.ZS,99.236641
EG.ELC.FOSL.ZS,EG.ELC.FOSL.ZS,100.000000
EG.ELC.HYRO.ZS,EG.ELC.HYRO.ZS,99.236641
EG.ELC.NGAS.ZS,EG.ELC.NGAS.ZS,99.236641


Now that we know the percentage of missing values in the 2016 columns, we can safely drop columns with a high percentage missing.

In [103]:
thresh = len(data_2016) * .9
data_2016.dropna(thresh = thresh, axis = 1, inplace = True)

In [104]:
data_2016

,Year,Country Name,Country Code,AG.LND.AGRI.K2,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EN.ATM.CO2E.GF.KT,EN.ATM.CO2E.GF.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.LF.KT,EN.ATM.CO2E.LF.ZS,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD,EN.ATM.CO2E.SF.KT,EN.ATM.CO2E.SF.ZS,EN.POP.DNST,NY.GDP.TOTL.RT.ZS,SP.RUR.TOTL,SP.URB.TOTL
1179,2016,Albania,ALB,11817.0,27400.0,28750.0,161.348,3.556993,4536.079,3362.639,74.130962,1.577163,0.130573,14.668,0.323363,104.967190,1.333387,1195854.0,1680247.0
1180,2016,Algeria,DZA,413602.0,2381741.0,2381741.0,78741.491,52.492238,150005.969,53519.865,35.678490,3.699156,0.318225,0.000,0.000000,17.025950,12.032482,11573776.0,28977628.0
1181,2016,Angola,AGO,558423.8,1246700.0,1246700.0,1496.136,4.312441,34693.487,17931.630,51.685868,1.202861,0.169340,0.000,0.000000,23.135064,10.857114,10340319.0,18502165.0
1182,2016,Argentina,ARG,1487000.0,2736690.0,2780400.0,102268.963,50.792234,201347.636,88792.738,44.099221,4.619085,0.227453,3010.607,1.495228,15.928135,0.936008,3649822.0,39940546.0
1183,2016,Armenia,ARM,16768.0,28470.0,29740.0,4191.381,81.294452,5155.802,828.742,16.073969,1.755976,0.164045,3.667,0.071124,103.131226,0.221818,1083966.0,1852180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,2016,"Venezuela, RB",VEN,215000.0,882050.0,912050.0,48796.769,29.722365,164175.257,76973.997,46.885261,5.500713,NaN,469.376,0.285899,33.837287,NaN,3532295.0,26313884.0
1306,2016,Vietnam,VNM,121722.0,310070.0,331230.0,20384.853,10.580309,192667.847,52559.111,27.279648,2.057566,0.313028,81187.380,42.138520,301.992208,3.355316,61324000.0,32314724.0
1307,2016,"Yemen, Rep.",YEM,233858.0,527970.0,527970.0,986.423,9.298306,10608.631,8104.070,76.391289,0.390480,NaN,322.696,3.041825,51.457867,0.797530,17552294.0,9615916.0
1308,2016,Zambia,ZMB,238360.0,743390.0,752610.0,0.000,0.000000,5141.134,2940.934,57.203994,0.314183,0.092280,854.411,16.619116,22.012009,11.680386,9419162.0,6944345.0


In [105]:
data_till_2016

,Year,Country Name,Country Code,AG.LND.AGRI.K2,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EG.ELC.COAL.ZS,EG.ELC.FOSL.ZS,EG.ELC.HYRO.ZS,EG.ELC.NGAS.ZS,...,EN.ATM.CO2E.SF.ZS,EN.CO2.BLDG.ZS,EN.CO2.ETOT.ZS,EN.CO2.MANF.ZS,EN.CO2.OTHX.ZS,EN.CO2.TRAN.ZS,EN.POP.DNST,NY.GDP.TOTL.RT.ZS,SP.RUR.TOTL,SP.URB.TOTL
0,2007,Albania,ALB,11190.00,27400.0,28750.0,0.000000,2.517483,97.482517,0.000000,...,1.867414,7.407407,8.730159,16.931217,11.640212,55.555556,108.394781,1.761086,1517619.0,1452398.0
1,2007,Algeria,DZA,412520.00,2381740.0,2381740.0,0.000000,99.392408,0.607592,97.257770,...,2.755153,16.028285,40.011786,11.455510,0.000000,32.516205,14.345383,32.464934,11839539.0,22327433.0
2,2007,Angola,AGO,508468.17,1246700.0,1246700.0,0.000000,22.381100,77.618900,0.000000,...,0.000000,23.858921,12.344398,23.547718,0.311203,40.041494,16.768559,46.279012,8879762.0,12025601.0
3,2007,Argentina,ARG,1440330.00,2736690.0,2780400.0,2.179398,63.438998,28.270726,51.046874,...,2.091227,14.616174,33.816513,20.510962,5.405569,25.644742,14.500837,4.995813,3823185.0,35861110.0
4,2007,Armenia,ARM,17519.00,28470.0,29740.0,0.000000,25.245846,31.417430,25.245846,...,0.145033,22.291667,22.500000,24.375000,8.750000,22.083333,103.007306,0.009296,1063397.0,1869221.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,2015,"Venezuela, RB",VEN,215000.00,882050.0,912050.0,0.000000,36.304958,63.695042,19.419168,...,0.300794,NaN,NaN,NaN,NaN,NaN,34.104449,NaN,3563493.0,26518336.0
1175,2015,Vietnam,VNM,121478.00,310070.0,331230.0,29.571446,63.267942,36.613975,33.210467,...,44.209276,NaN,NaN,NaN,NaN,NaN,298.890818,4.206113,61343883.0,31333193.0
1176,2015,"Yemen, Rep.",YEM,234326.00,527970.0,527970.0,0.000000,100.000000,0.000000,39.804732,...,2.700445,NaN,NaN,NaN,NaN,NaN,50.188247,2.105149,17282718.0,9215171.0
1177,2015,Zambia,ZMB,238360.00,743390.0,752610.0,0.000000,3.006176,96.993824,0.000000,...,10.679612,NaN,NaN,NaN,NaN,NaN,21.360741,11.163582,9224797.0,6654564.0


In [116]:
thresh = len(data_till_2016) * .9
data_till_2016.dropna(thresh = thresh, axis = 1, inplace = True)
data_till_2016

cols = list(data_2016.columns)
cols.remove("Year")
cols.remove("Country Code")
cols.remove("Country Name")
gg_2016 = data_2016.groupby("Country Name").agg({i: "sum" for i in cols})
agg_2016

,AG.LND.AGRI.K2,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EN.ATM.CO2E.GF.KT,EN.ATM.CO2E.GF.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.LF.KT,EN.ATM.CO2E.LF.ZS,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD,EN.ATM.CO2E.SF.KT,EN.ATM.CO2E.SF.ZS,EN.POP.DNST,NY.GDP.TOTL.RT.ZS,SP.RUR.TOTL,SP.URB.TOTL
Country Name,,,,,,,,,,,,,,,,
Albania,11817.0,27400.0,28750.0,161.348,3.556993,4536.079,3362.639,74.130962,1.577163,0.130573,14.668,0.323363,104.967190,1.333387,1195854.0,1680247.0
Algeria,413602.0,2381741.0,2381741.0,78741.491,52.492238,150005.969,53519.865,35.678490,3.699156,0.318225,0.000,0.000000,17.025950,12.032482,11573776.0,28977628.0
Angola,558423.8,1246700.0,1246700.0,1496.136,4.312441,34693.487,17931.630,51.685868,1.202861,0.169340,0.000,0.000000,23.135064,10.857114,10340319.0,18502165.0
Argentina,1487000.0,2736690.0,2780400.0,102268.963,50.792234,201347.636,88792.738,44.099221,4.619085,0.227453,3010.607,1.495228,15.928135,0.936008,3649822.0,39940546.0
Armenia,16768.0,28470.0,29740.0,4191.381,81.294452,5155.802,828.742,16.073969,1.755976,0.164045,3.667,0.071124,103.131226,0.221818,1083966.0,1852180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, RB",215000.0,882050.0,912050.0,48796.769,29.722365,164175.257,76973.997,46.885261,5.500713,0.000000,469.376,0.285899,33.837287,0.000000,3532295.0,26313884.0
Vietnam,121722.0,310070.0,331230.0,20384.853,10.580309,192667.847,52559.111,27.279648,2.057566,0.313028,81187.380,42.138520,301.992208,3.355316,61324000.0,32314724.0
"Yemen, Rep.",233858.0,527970.0,527970.0,986.423,9.298306,10608.631,8104.070,76.391289,0.390480,0.000000,322.696,3.041825,51.457867,0.797530,17552294.0,9615916.0


In [117]:
new_cols = list(data_2016.columns)
new_cols.remove("Year")
new_cols.remove("Country Code")
new_cols.remove("Country Name")
agg_2016 = data_2016.groupby("Country Name").agg({i: "sum" for i in new_cols})
agg_2016

,AG.LND.AGRI.K2,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EN.ATM.CO2E.GF.KT,EN.ATM.CO2E.GF.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.LF.KT,EN.ATM.CO2E.LF.ZS,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD,EN.ATM.CO2E.SF.KT,EN.ATM.CO2E.SF.ZS,EN.POP.DNST,NY.GDP.TOTL.RT.ZS,SP.RUR.TOTL,SP.URB.TOTL
Country Name,,,,,,,,,,,,,,,,
Albania,11817.0,27400.0,28750.0,161.348,3.556993,4536.079,3362.639,74.130962,1.577163,0.130573,14.668,0.323363,104.967190,1.333387,1195854.0,1680247.0
Algeria,413602.0,2381741.0,2381741.0,78741.491,52.492238,150005.969,53519.865,35.678490,3.699156,0.318225,0.000,0.000000,17.025950,12.032482,11573776.0,28977628.0
Angola,558423.8,1246700.0,1246700.0,1496.136,4.312441,34693.487,17931.630,51.685868,1.202861,0.169340,0.000,0.000000,23.135064,10.857114,10340319.0,18502165.0
Argentina,1487000.0,2736690.0,2780400.0,102268.963,50.792234,201347.636,88792.738,44.099221,4.619085,0.227453,3010.607,1.495228,15.928135,0.936008,3649822.0,39940546.0
Armenia,16768.0,28470.0,29740.0,4191.381,81.294452,5155.802,828.742,16.073969,1.755976,0.164045,3.667,0.071124,103.131226,0.221818,1083966.0,1852180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, RB",215000.0,882050.0,912050.0,48796.769,29.722365,164175.257,76973.997,46.885261,5.500713,0.000000,469.376,0.285899,33.837287,0.000000,3532295.0,26313884.0
Vietnam,121722.0,310070.0,331230.0,20384.853,10.580309,192667.847,52559.111,27.279648,2.057566,0.313028,81187.380,42.138520,301.992208,3.355316,61324000.0,32314724.0
"Yemen, Rep.",233858.0,527970.0,527970.0,986.423,9.298306,10608.631,8104.070,76.391289,0.390480,0.000000,322.696,3.041825,51.457867,0.797530,17552294.0,9615916.0
